In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re

from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import string
import os
import time


In [3]:
options = Options()
#options.headless = False
#options.add_argument("--window-size=1920,1200")

# This is the primary dictionary with the links of the phases.  We will add to it all the info'
course_info = [
    {'phase_title':'Pre-work', 'phase_link':'https://learning.flatironschool.com/courses/6103'},
]

In [4]:
def getPrimaryData():

    print("Logging in...")
    driver = webdriver.Chrome(options=options)
    driver.get("https://portal.flatironschool.com/sign_in")
    time.sleep(1)

    email = driver.find_elements(By.NAME, 'user[email]')
    pw = driver.find_elements(By.NAME, 'user[password]')
    login = driver.find_elements(By.NAME, 'commit')

    email[0].send_keys('natekist@outlook.com')
    pw[0].send_keys('jfq@zhb6wub*KCQ*hxd')
    login[0].click()
    
    # Loop through the phases 
    for n in course_info:
        
        # Make a phase_data item to be later added to course_info dictionary
        n['phase_data'] = []

        # Get and parse the html source on the phase page
        print("Getting ", n['phase_title'], "...", sep="")
 
        driver.get(n['phase_link'])
        time.sleep(.5)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        regex = re.compile("item-group-condensed context_module*")
        col_headings = soup.find_all('div', {'class':regex})

        # Loop through the collapsible headings
        for index, val in enumerate(col_headings):
            
            # Get the title of the heading and get all the links under it
            col_heading_title = val.attrs["aria-label"]
            links = val.find_all('a', {'class':'ig-title title item_link'})

            # Make sure it's not a heading with zero links
            if len(links) > 0:
                col_heading_title = "".join(i for i in col_heading_title if i in string.printable).strip()
            else:
                continue

            # Create a variable for new heading to be added
            new_heading = {
                "heading_title":col_heading_title,
                'heading_data':[]
            }

            # Loop through the links in the collapsible heading
            for t in links:
                title = t.attrs['title']
                href = t.attrs['href']
                full_url = urljoin(n['phase_link'], href)
                title = "".join(i for i in title if i in string.printable).strip()
                new_link = {
                    'link_title':title,
                    'link_url':full_url,
                    'link_git':"",
                    'link_vim':"",
                    'link_done':0
                }
                new_heading['heading_data'].append(new_link)

            # Add the new heading to the phase list item
            n['phase_data'].append(new_heading)
    print("Finished getting primary course data.")

def save_info():
    with open('course_info.json', 'w') as f:
        json.dump(course_info, f)
        
# Prints out the phase/heading/links - print it in order of 1) primary content, 2) lecture, 3) exit tickets, 4) quizes
def print_course_info():
    
    for i in course_info:
        # Print the phase name
        print(f"\n\n{i['phase_title']}")
        
        for j in i['phase_data']:
            print(f"\n{j['heading_title']}\n")
            
            for l in j['heading_data']:
                if 'Quiz' not in l['link_title'] and 'Exit Ticket' not in l['link_title'] and 'Lecture' not in l['link_title']: 
                    print(f" - {l['link_title']}")
                    
            for l in j['heading_data']:
                if 'Lecture' in l['link_title']: 
                    print(f" - {l['link_title']}")
                    
            for l in j['heading_data']:
                if 'Exit Ticket' in l['link_title']: 
                    print(f" - {l['link_title']}")
            
            for l in j['heading_data']:
                if 'Quiz' in l['link_title']: 
                    print(f" - {l['link_title']}")
            

In [8]:
# Check how many links there are
count_total = 0
count_done = 0

for i in course_info:
    for j in i['phase_data']:
        for l in j['heading_data']:
            count_total+=1
            if l['link_done'] !=0:
                count_done +=1

count_not_done_at_start = count_total - count_done
print(f"There are {count_total} total lesson links, of which {count_not_done_at_start} still need to be checked for external links.\n")

if (count_total - count_done) > 0:
    print("Logging in...")
    driver = webdriver.Chrome(options=options)
    driver.get("https://portal.flatironschool.com/sign_in")
    time.sleep(1)

    email = driver.find_elements(By.NAME, 'user[email]')
    pw = driver.find_elements(By.NAME, 'user[password]')
    login = driver.find_elements(By.NAME, 'commit')

    email[0].send_keys('natekist@outlook.com')
    pw[0].send_keys('jfq@zhb6wub*KCQ*hxd')
    login[0].click()

    count_done = 0
    for i in course_info:
        for j in i['phase_data']:
            for l in j['heading_data']:
                if l['link_done'] == 0:
                    count_done+=1
                    print(f"{count_done:>3}/{count_not_done_at_start:>}: Checking {l['link_title']} ", sep="", end="")
                    driver.get(l['link_url'])
                    for _ in range(1, 4):
                        time.sleep(1)
                        source = driver.page_source
                        # every 3 seconds check for github link, max of 15 seconds waiting
                        start = source.find("https://github.com/learn-co-curriculum")
                        if start != -1:
                            end = source.find('"', start) 
                            l['link_git'] = (source[start:end].strip())
                            print("[GH]", end="")
                            break
                        start = source.find("https://player.vimeo.com/")
                        if start != -1:
                            end = source.find('?', start) 
                            l['link_vim'] = (source[start:end].strip())
                            print("[VIM]", end="")
                            break
                    l['link_done'] = 1
                    print("")
                    save_info()            




                
                    
                
                
                
                
                
                
                

There are 52 total lesson links, of which 0 still need to be checked for external links.



Load the course info from file

In [11]:
if os.path.exists('course_info.json'):
    with open('course_info.json', 'r') as f:
        course_info = json.load(f)

    print("Saved course info found/loaded.\n")
else:
    print("No saved course info.  Getting primary course data...\n")
    getPrimaryData()
    save_info()
    print("Course info saved to file.")

Saved course info found/loaded.



View what is in course_info

In [17]:
course_info

[{'phase_title': 'Pre-work',
  'phase_link': 'https://learning.flatironschool.com/courses/6103',
  'phase_data': [{'heading_title': 'Introduction',
    'heading_data': [{'link_title': 'Welcome to Prep!',
      'link_url': 'https://learning.flatironschool.com/courses/6103/modules/items/470895',
      'link_git': 'https://github.com/learn-co-curriculum/dsc-prework-intro\\',
      'link_vim': '',
      'link_done': 1},
     {'link_title': 'The Structure of This Course',
      'link_url': 'https://learning.flatironschool.com/courses/6103/modules/items/470896',
      'link_git': '',
      'link_vim': '',
      'link_done': 1},
     {'link_title': 'Your First Jupyter Notebook!',
      'link_url': 'https://learning.flatironschool.com/courses/6103/modules/items/470897',
      'link_git': 'https://github.com/learn-co-curriculum/dsc-prework-jupyter-notebook\\',
      'link_vim': '',
      'link_done': 1}]},
   {'heading_title': 'Prep Pre-test',
    'heading_data': [{'link_title': 'Prep Pre-Test'

The following function saves the html text into a file with collapsible headings.  

In [13]:
def save_html_file(file_path):
    all_text_begin = '''
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
    .collapsible {
      background-color: #777;
      color: white;
      cursor: pointer;
      padding: 3px;
      width: 100%;
      border: none;
      text-align: left;
      outline: none;
      font-size: 15px;
    }

    .active, .collapsible:hover {
      background-color: #555;
    }

    .content {
      padding: 0 18px;
      display: none;
      overflow: hidden;
      background-color: #f1f1f1;
    }
    </style>
    </head>
    <body>
    '''

    all_text_end = '''
    <script>
    var coll = document.getElementsByClassName("collapsible");
    var i;

    for (i = 0; i < coll.length; i++) {
      coll[i].addEventListener("click", function() {
        this.classList.toggle("active");
        var content = this.nextElementSibling;
        if (content.style.display === "block") {
          content.style.display = "none";
        } else {
          content.style.display = "block";
        }
      });
    }
    </script>

    </body>
    </html>

    '''

    output_file_obj = open(file_path, mode='w')
    output_file_obj.write(all_text_begin + all_text + all_text_end)
    output_file_obj.close()

The cell below run through the course phases and lessons and puts all links (FIS, GH, VIM) next to each.  It includes all course items.  

In [ ]:
all_text = ""
for i in course_info:
    phase_title = i['phase_title']
    
    # Print the phase name
    print(f"\n\n{phase_title}")
    all_text = all_text + '<button type="button" class="collapsible"><h3>' + phase_title + '</h3></button><div class="content"><br>'

    for j in i['phase_data']:
        heading_title = j['heading_title']
        print(f"\n{heading_title}\n")
        all_text = all_text + "<b>" + heading_title + "</b><ul>"

        for l in j['heading_data']:
            print(f" - {l['link_title']}")
            if l['link_git'] == "" and l['link_vim'] == "":
                all_text = all_text + '<li>' + l['link_title'] + '  <a href="' + l['link_url'] + '"' + ' target="blank_">' + '[FIS]' + '</a></li>'
            elif l['link_git'] != "":
                all_text = all_text + '<li>' + l['link_title']  + '  <a href="' + l['link_url'] + '"' + ' target="blank_">' + '[FIS]' + '</a>' + '  <a href="' + l['link_git'] + '"' + ' target="blank_">' + '[GH] ' + '</a></li>'
            else:
                all_text = all_text + '<li>' + l['link_title']  + '  <a href="' + l['link_url'] + '"' + ' target="blank_">' + '[FIS]' + '</a>' + '  <a href="' + l['link_vim'] + '"' + ' target="blank_">' + '[VIM] ' + '</a></li>'
        all_text = all_text + "</ul>"   
    all_text = all_text + "</div>"
save_html_file("DS_Links_All.html")

The cell below takes course_info dictionary and turns it into a collapsible HTML file.  It ignores internal internal Flatiron school items like quizzes and exit tickets, and does not include Flatiron links (assumes course access is expired).  

In [18]:
all_text = ""
for i in course_info:
    phase_title = i['phase_title']
    
    # Print the phase name
    #print(f"\n\n{phase_title}")
    all_text = all_text + '<button type="button" class="collapsible"><h3>' + phase_title + '</h3></button><div class="content"><br>'

    for j in i['phase_data']:
        heading_title = j['heading_title']
        #print(f"\n{heading_title}\n")
        all_text = all_text + "<b>" + heading_title + "</b><ul>"

        for l in j['heading_data']:
            if 'Quiz' in l['link_title'] or 'Exit Ticket' in l['link_title'] or 'Priorities' in l['link_title']:
                continue
            else:
                #print(f" - {l['link_title']}")
                if l['link_git'] == "" and l['link_vim'] == "":
                    all_text = all_text + '<li>' + l['link_title']
                elif l['link_git'] != "":
                    all_text = all_text + '<li>' + l['link_title'] + '  <a href="' + l['link_git'] + '"' + ' target="blank_">' + '[GH] ' + '</a></li>'
                else:
                    all_text = all_text + '<li>' + l['link_title']  + '  <a href="' + l['link_vim'] + '"' + ' target="blank_">' + '[VIM] ' + '</a></li>'
        all_text = all_text + "</ul>"   
    all_text = all_text + "</div>"
save_html_file("DS_Links.html")